In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
import torch
import torch.nn as nn
import torch.optim as optim
import emoji
import re
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup

In [ ]:
# model_name = "sentence-transformers/bert-base-nli-mean-tokens"
model_name = "cardiffnlp/twitter-roberta-base"
# model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
annotated = pd.read_csv('../input/annotated-clean/annotated_clean.csv')
annotated.head(5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
labeled_tweets = annotated.rename({'tweet_text': 'text'}, axis=1)
labeled_tweets['label'] = labeled_tweets['label']-1

train_size = 0.8
train_data=labeled_tweets.sample(frac=train_size,random_state=200)
test_data=labeled_tweets.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(labeled_tweets.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

In [ ]:
def encode_sentences(sentences):
    encoded = tokenizer(sentences, padding=True, return_attention_mask=True, return_tensors='pt')
    return encoded

def collate_batch(batch):
    sentences, targets = list(zip(*batch))
    encoded = encode_sentences(list(sentences))
    targets = torch.tensor(targets)
    return encoded, targets

class TweetDataset(torch.utils.data.Dataset):    
    def __init__(self, df):
        self.df = df.text.to_list()
        self.targets = df.label.to_list()
     
    def __getitem__(self, idx):
        sentence = self.df[idx]
        target = self.targets[idx]
        return sentence, target
     
    def __len__(self):
        return len(self.df)

In [ ]:

import multiprocessing
train_dataset = TweetDataset(train_data)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=collate_batch, num_workers=(multiprocessing.cpu_count() - 1), pin_memory=True)

test_dataset = TweetDataset(test_data)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, collate_fn=collate_batch, num_workers=(multiprocessing.cpu_count() - 1), pin_memory=True)

In [ ]:
class Model(nn.Module):
    def __init__(self, model_name, num_classes):
        super(Model, self).__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size
        self.classify = nn.Sequential(
            nn.LayerNorm(hidden_size),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, num_classes)
        )
        
    def mean_pooling(self, outputs, attention_mask):
        token_embeddings = outputs[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    def forward(self, inputs):
        outputs = self.encoder(**inputs)
        return self.classify(outputs[1])

In [ ]:
model = Model(model_name, 3)
model = model.to(device)

In [ ]:
EPOCHS = 5
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), betas = (0.99, 0.98), lr=2e-5)
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

In [ ]:
def train(model, dataloader, loss_fn, optimizer, scheduler):
    model = model.train()
    losses = []
    correct_predictions = 0
    for sentences, targets in tqdm(dataloader):
        input_ids = sentences["input_ids"].to(device)
        attention_mask = sentences["attention_mask"].to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()        
        outputs = model(dict(
            input_ids=input_ids,
            attention_mask=attention_mask
        ))
        
        train_loss = loss_fn(outputs, targets)
        train_loss.backward()
        losses.append(train_loss.item())
        
        _, preds = torch.max(outputs, dim=1)
        correct_predictions += torch.sum(preds == targets)
        
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
    return correct_predictions.double() / len(dataloader.dataset), np.mean(losses)

In [ ]:
def validate(model, dataloader, loss_fn):
    model = model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for sentences, targets in tqdm(dataloader):
            input_ids = sentences["input_ids"].to(device)
            attention_mask = sentences["attention_mask"].to(device)
            targets = targets.to(device)
            
            outputs = model(dict(
                input_ids=input_ids,
                attention_mask=attention_mask
            ))
            
            val_loss = loss_fn(outputs, targets)
            losses.append(val_loss.item())
            
            _, preds = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(preds == targets)
            
    return correct_predictions.double() / len(dataloader.dataset), np.mean(losses)

In [ ]:
for epoch in range(EPOCHS):
    print(f"Epoch: {epoch + 1} / {EPOCHS}")
    
    train_accuracy, train_loss = train(model, train_dataloader, loss_fn, optimizer, scheduler)
    val_accuracy, val_loss = validate(model, test_dataloader, loss_fn)
    
    print(f"Training Loss: {train_loss} | Training Accuracy: {train_accuracy}")
    print(f"Validation Loss: {val_loss} | Validation Accuracy: {val_accuracy}")  

In [ ]:
all_tweet = pd.read_csv("../input/all-tweets-cleaned/all_tweets_cleaned.csv")
all_tweet['label'] = 0
all_tweet.head(5)

In [ ]:

def predict(model, dataloader):
    prediction_list = []
    model = model.eval()
    with torch.no_grad():
        for sentences, targets in tqdm(dataloader):
            input_ids = sentences["input_ids"].to(device)
            attention_mask = sentences["attention_mask"].to(device)
            targets = targets.to(device)
            
            outputs = model(dict(
                input_ids=input_ids,
                attention_mask=attention_mask
            ))
            
            predictions = np.argmax(outputs.cpu(), axis=1)
            prediction_list = prediction_list + predictions.tolist()
            
    return prediction_list

In [ ]:
test_dataset = TweetDataset(all_tweet)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, collate_fn=collate_batch, num_workers=(multiprocessing.cpu_count() - 1), pin_memory=True)

In [ ]:
prediction_list = predict(model, test_dataloader)

In [ ]:
all_tweet.head(5)

In [ ]:
all_tweet['label'] = prediction_list

In [ ]:
all_tweet.head(5)


In [ ]:
all_tweet['label'].value_counts()

In [ ]:
all_tweet.to_csv("all_tweet_labeled.csv")